# async, await

ref :
- http://postd.cc/python-generators-coroutines-native-coroutines-and-async-await/
- http://blanktar.jp/blog/2015/07/python-yield-from.html

## setting

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

## generater

In [3]:
def generate_nums():
    num = 0
    while True:
        yield num
        print("Num: ", num)
        num = num + 1
 
 
nums = generate_nums()
 
for x in nums:
    print(x)
    print("="*9)
 
    if x > 9:
        break

0
Num:  0
1
Num:  1
2
Num:  2
3
Num:  3
4
Num:  4
5
Num:  5
6
Num:  6
7
Num:  7
8
Num:  8
9
Num:  9
10


In [7]:
def coro():
    hello = yield "Hello"
    yield hello
 
c = coro()

In [8]:
print(next(c))

Hello


In [67]:
print(c.send("World"))

World


### ジェネレーターとメモリ消費量
ジェネレーターはメモリ消費量が少なくすむ。
リスト内包表記でもしかり

In [7]:
from memory_profiler import profile

In [6]:
def tmp():
    print("hoge")

In [7]:
%load_ext memory_profiler

In [20]:
iter_num = 10000000

In [21]:
%memit
len([x for x in range(iter_num) if x % 3 == 0])

peak memory: 44.48 MiB, increment: 0.00 MiB


3333334

# Coroutine
コルーチンは、ジェネレータ関数を呼び出した後、新しい情報をその関数に与えることができる
ref. https://www.kannon.link/fuku/index.php/2016/10/16/01-14/

ざっくり言うと、一時停止と再開が可能で外からコントロールできる関数がコルーチンです。  
ref. https://www.kannon.link/fuku/index.php/2016/11/14/01-21/

In [50]:
def simple_coroutine(x=3):
    print("-> start: ", x)
    x = yield
    print("-> received: ", x)

In [62]:
si_coro = simple_coroutine()

In [63]:
si_coro.send(None)

-> start:  3


In [64]:
si_coro.send(10)

-> received:  10


StopIteration: 

In [69]:
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        print("average: ", average)
        print("term: ", term)
        total += term
        count += 1
        average = total/count

In [70]:
avg=averager()

In [71]:
avg.send(None) # 1回目はNone以外をSendできない

In [72]:
avg.send(10)

average:  None
term:  10


10.0

In [73]:
avg.send(20)

average:  10.0
term:  20


15.0

In [74]:
avg.send(30)

average:  15.0
term:  30


20.0

In [22]:
%memit
sum((1 for x in range(iter_num) if x % 3 == 0))

peak memory: 47.77 MiB, increment: 0.00 MiB


3333334

## what is 'yield from'
大きい関数を小さい関数に分割するのと同じように、ジェネレータも小さく分割できるようにするために導入されたもの。
ref. https://www.kannon.link/fuku/index.php/2016/11/12/01-20/

1.  yield from は後ろに Iterable (sub generator) をとり、そこから Iterator を取り出す (.__iter__() を呼び出す)
2.  Iterator を進めて、出てきた要素を外側 (generator を進めている側) に戻す
3.  外側が generator を進める (.__next__() や .send() を呼び出す) と、 Iterator の止まっていた部分から動作を再開する
4.  Iterator が sub generator の場合、そこから return された値が yield from の結果として generator に渡る

ref. http://u7fa9.org/memo/HEAD/archives/2016-06/2016-06-25.rst

In [4]:
def range_ten():
    #yield from range(10)
    num = 0
    while True:
        yield num
        if num == 10:
            break
        num += 1
        
def range_ten_from():
    yield from range(10)

In [5]:
x = range_ten()
for i in x:
    if i == 5:
        break
    print(i)

0
1
2
3
4


In [7]:
for i in range_ten_from():
    print(i)

0
1
2
3
4
5
6
7
8
9


In [54]:
next(x)

4

In [55]:
next(x)

5

In [8]:
def tea():
    yield from range(0,4)
    return "Hello"

In [33]:
tea()

<generator object tea at 0x1043ade08>

In [30]:
def teb():
    x = yield from tea()
    print(x)

In [34]:
teb()

<generator object teb at 0x104374c50>

In [25]:
x = tea()

In [22]:
for i in x:
    print(i)

0
1
2
3


In [31]:
tuple(teb())

Hello


(0, 1, 2, 3)

In [31]:
tuple(teb())

Hello


(0, 1, 2, 3)

# async / await(python >= 3.5)

- async/await は yield from と動作は同じだが、
- def の代わりに async def, yield from の代わりに await と書く
- Iterable の代わりに Awaitable をとる
- Awaitable は .__iter__() の代わりに .__await__() が呼ばれる (.__await__() が返すものは、 .__iter__() と同様 Iterator でよい。)
- (呼び出し結果の coroutine は __iter__, __next__ を持たない (Iterable/Iterator ではない))
という違いがある。
ref. http://u7fa9.org/memo/HEAD/archives/2016-06/2016-06-25.rst

コルーチン関数は新たな構文 async def を用いて定義されます
…
コルーチン関数内で、新たな await 式を用いることで結果が利用可能になるまでコルーチンの実行を停止することが出来ます。  
ref. http://docs.python.jp/3/whatsnew/3.5.html#whatsnew-pep-492

Pythonの中でのコルーチンのコンセプトをはっきりさせ、非同期プログラミングをできるだけ意識しないで書けるようにすることが目的とのことです。
今までは、ジェネレータもコルーチンも同じyieldキーワード使っていたので、両者を混同しやすかったのもこの構文導入のきっかけのようです。  
ref. https://www.kannon.link/fuku/index.php/2016/11/14/01-21/


In [80]:
import asyncio
import random

async def producer(num, queue):
    while True:
        x = random.randint(1, 100)
        print("producer-" + str(num) + " : " + str(x))
        await queue.put(x)
        await asyncio.sleep(10)

async def consumer(num, queue):
    while True:
        value = await queue.get()
        print("consumer-" + str(num) + " : " + str(value))

In [79]:
loop = asyncio.get_event_loop()
q = asyncio.Queue()
loop.create_task(producer(1, q))
loop.create_task(producer(2, q))
loop.create_task(consumer(1, q))
loop.run_forever()

producer-1 : 89
producer-2 : 47
consumer-1 : 89
consumer-1 : 47
producer-1 : 100
producer-2 : 32
consumer-1 : 100
consumer-1 : 32
producer-1 : 40
producer-2 : 38
consumer-1 : 40
consumer-1 : 38
producer-1 : 55
producer-2 : 45
consumer-1 : 55
consumer-1 : 45


KeyboardInterrupt: 

# asyncio (python3.4~)

In [69]:
import asyncio
import datetime
import random
 
 
@asyncio.coroutine
def display_date(num, loop):
    end_time = loop.time() + 10.0
    while True:
        print("Loop: {} Time: {}".format(num, datetime.datetime.now()))
        if (loop.time() + 1.0) >= end_time:
            break
        yield from asyncio.sleep(random.randint(0, 5))
 
 
loop = asyncio.get_event_loop()
 
asyncio.ensure_future(display_date(1, loop))
asyncio.ensure_future(display_date(2, loop))
 
loop.run_forever()

Loop: 1 Time: 2016-12-20 17:25:49.858244
Loop: 2 Time: 2016-12-20 17:25:49.858428
Loop: 1 Time: 2016-12-20 17:25:49.858551
Loop: 2 Time: 2016-12-20 17:25:49.858612
Loop: 1 Time: 2016-12-20 17:25:49.858651
Loop: 1 Time: 2016-12-20 17:25:51.859859
Loop: 2 Time: 2016-12-20 17:25:53.863919
Loop: 1 Time: 2016-12-20 17:25:53.864109
Loop: 2 Time: 2016-12-20 17:25:53.864237
Loop: 2 Time: 2016-12-20 17:25:56.869681
Loop: 1 Time: 2016-12-20 17:25:58.865731
Loop: 2 Time: 2016-12-20 17:26:01.875245


KeyboardInterrupt: 

In [113]:
print(next(display_date(2, loop)))

Loop: 2 Time: 2016-12-20 17:31:00.023066
<Future pending>


In [117]:
num =0
for i in asyncio.sleep(0.1):
    if num == 10:
        break
    print(num, ": ", i)

0 :  <Future pending>


AssertionError: yield from wasn't used with future

In [118]:
asyncio.sleep(0.1)

<generator object sleep at 0x1046c1fc0>